In [1]:
# bob.refresh {"cron": "5,30 2-12 * * *", "timeout": 40000}

In [2]:
%run __init__.ipynb

Successfully connected to MongoDB


In [13]:
hc = bob.HealthCheck()
hc_key = "51c8f3e8-3d84-4888-878e-a53f0a152e1a"
if use_healthchecks:
    step_1 = hc.start(hc_key)
    print(step_1)

HealthCheck init in prod mode
Start ==> send to https://health.cashstory.com/ping/51c8f3e8-3d84-4888-878e-a53f0a152e1a, 2020-04-28


In [16]:
#--- Check if the data are already updated
filePath = f'{output_folder}WORLD_DB_TREND.csv'
if path.exists(filePath):
    df_check = pd.read_csv(filePath, sep=';',low_memory=False)
    date_check = df_check[df_check['SCENARIO'] == yesterday.strftime('%d/%m/%Y')]
    if len(date_check) == 0:
        %run "covid-19_world.ipynb"
    else:
        if use_healthchecks:
            step_2 = hc.done(hc_key)
            print(step_2)
            raise Exception(f'Data already updated ! DateTime : {now}')
else:
    %run "covid-19_world.ipynb"

End ==> send to https://health.cashstory.com/ping/51c8f3e8-3d84-4888-878e-a53f0a152e1a, 2020-04-28


SystemExit: Data already updated ! DateTime : 28/04/2020 06:47:01

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [3]:
#--- Load data source
start_time = time.time()
db_conso = get_datasource('WORLD_DB_CONSO')
db_all = get_datasource('WORLD_DB_ALL')
db_trend = get_datasource('WORLD_DB_TREND')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))

Script execution completed at 29/04/2020 15:38:23. Time: --- 3.4959702491760254 secnds ---


In [19]:
#-- Date selector : ENTITIES
def get_entities(df):
    cols_to_rename = {"ENTITY": "ENTITIES", "ENTITY_GROUPS": "GROUPS"}
    filters = ['WORLDWIDE','Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    df = df[df['ENTITY_GROUPS'].isin(filters)].rename(index=str, columns=cols_to_rename)[['ENTITIES','GROUPS']].drop_duplicates().sort_values(by="ENTITIES", ascending=True).reset_index(drop=True)
    df['ORDER'] = df.index + 8
    df.loc[df['ENTITIES'] == 'WORLDWIDE', 'ORDER'] = 0
    df.loc[df['ENTITIES'] == 'Europe', 'ORDER'] = 1
    df.loc[df['ENTITIES'] == 'Asia', 'ORDER'] = 2
    df.loc[df['ENTITIES'] == 'Africa', 'ORDER'] = 3
    df.loc[df['ENTITIES'] == 'North America', 'ORDER'] = 4
    df.loc[df['ENTITIES'] == 'South America', 'ORDER'] = 5
    df.loc[df['ENTITIES'] == 'Oceania', 'ORDER'] = 6
    df.loc[df['ENTITIES'] == 'Other', 'ORDER'] = 7
    df.loc[df['ENTITIES'] == 'To be affected', 'ORDER'] = 8
    
    df.loc[df['ENTITIES'] == 'WORLDWIDE', 'ORDER_GROUPS'] = 0
    df.loc[df['ENTITIES'] == 'Europe', 'ORDER_GROUPS'] = 1
    df.loc[df['ENTITIES'] == 'Asia', 'ORDER_GROUPS'] = 2
    df.loc[df['ENTITIES'] == 'Africa', 'ORDER_GROUPS'] = 3
    df.loc[df['ENTITIES'] == 'North America', 'ORDER_GROUPS'] = 4
    df.loc[df['ENTITIES'] == 'South America', 'ORDER_GROUPS'] = 5
    df.loc[df['ENTITIES'] == 'Oceania', 'ORDER_GROUPS'] = 6
    df.loc[df['ENTITIES'] == 'Other', 'ORDER_GROUPS'] = 7
    df.loc[df['ENTITIES'] == 'To be affected', 'ORDER_GROUPS'] = 8

    df.loc[df['GROUPS'] == 'Europe', 'ORDER_GROUPS'] = 1
    df.loc[df['GROUPS'] == 'Asia', 'ORDER_GROUPS'] = 2
    df.loc[df['GROUPS'] == 'Africa', 'ORDER_GROUPS'] = 3
    df.loc[df['GROUPS'] == 'North America', 'ORDER_GROUPS'] = 4
    df.loc[df['GROUPS'] == 'South America', 'ORDER_GROUPS'] = 5
    df.loc[df['GROUPS'] == 'Oceania', 'ORDER_GROUPS'] = 6
    df.loc[df['GROUPS'] == 'Other', 'ORDER_GROUPS'] = 7
    df.loc[df['GROUPS'] == 'To be affected', 'ORDER_GROUPS'] = 8
    
    df.loc[df['GROUPS'] == 'WORLDWIDE','ENTITY_DISPLAY'] = df['ENTITIES']
    df.loc[df['GROUPS'] != 'WORLDWIDE','ENTITY_DISPLAY'] = '--- ' + df['ENTITIES']
    return df.sort_values(by=['ORDER_GROUPS','ORDER'], ascending=True)

entities = get_entities(db_conso)
entities.to_csv(output_folder + 'ENTITIES.csv',sep=";")
if use_mongo:
    bob.mongo.save_df(entities,'ENTITIES',db_app,True)
# entities

Dataframe ENTITIES successfully save in database app-wsr-dev in MongoDB. Time: --- 0.023267745971679688 secnds ---


In [20]:
#-- Date selector : SCENARIOS
def get_scenarios(df):
    df = df[['SCENARIO','DATE_ORDER']].drop_duplicates()
    df['SCENARIO_DIPLAY'] = pd.to_datetime(df['SCENARIO'], format='%d/%m/%Y').dt.strftime('%d %b %Y')
    df['SCENARIO_TODAY'] = (pd.to_datetime(df['SCENARIO'], format='%d/%m/%Y') + timedelta(days = 1)).dt.strftime('%d/%m/%Y')
    df['SCENARIO_TODAY_DIPLAY'] = pd.to_datetime(df['SCENARIO_TODAY'], format='%d/%m/%Y').dt.strftime('%d %b %Y')
    return df.sort_values(by='DATE_ORDER', ascending=False)

scenarios = get_scenarios(db_all)
scenarios.to_csv(output_folder + 'SCENARIOS.csv',sep=";")
if use_mongo:
    bob.mongo.save_df(scenarios,'SCENARIOS',db_app,True)
# scenarios

Dataframe SCENARIOS successfully save in database app-wsr-dev in MongoDB. Time: --- 0.03566145896911621 secnds ---


In [21]:
#Dashboard value dynamic template
def dashboard_value(df,kpi,domain_num):
    filters = ['WORLDWIDE','Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    domain = df[(df['ENTITY_GROUPS'].isin(filters)) & (df['KPI'].isin([kpi]))]
    domain = pd.DataFrame({
        'ENTITY': domain['ENTITY'],
        'ENTITY_GROUPS': domain['ENTITY_GROUPS'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['KPI'],
        'VALUE': domain['VALUE'],
        'VALUE_D-1': domain['VALUE_D-1'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VALUE': domain['UNIT_VALUE'],
        'UNIT_VAR': domain['UNIT_VAR'],
        'UNIT_VARP': domain['UNIT_VARP'],
        'PRECISION_VALUE': domain['PRECISION_VALUE'],
        'PRECISION_VAR': domain['PRECISION_VAR'],
        'PRECISION_VARP': domain['PRECISION_VARP'],
        'LAST_UPDATE': now,
    })
    domain = domain.sort_values(by=['ENTITY','SCENARIO'],ascending=False)
    domain.to_csv(output_folder + domain_num + '.csv',sep=";")
    if use_mongo:
        bob.mongo.save_df(domain,domain_num, db_app,True)
    df_collection = MongoDB[db_app][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING),("SCENARIO", DESCENDING)])
    return domain

In [22]:
#--- DASHBOARD ---
#-- VALUE DYNAMIC KPIS
start_time = time.time()
domain001 = dashboard_value(db_all,'Confirmed','001')
domain002 = dashboard_value(db_all,'Deaths','002')
domain003 = dashboard_value(db_all,'Recovered','003')
domain004 = dashboard_value(db_all,'Active cases','004')
domain005 = dashboard_value(db_all,"Fatality Rate",'005')
domain006 = dashboard_value(db_all,"Recovery Rate",'006')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))

Dataframe 001 successfully save in database app-wsr-dev in MongoDB. Time: --- 13.520699739456177 secnds ---
Dataframe 002 successfully save in database app-wsr-dev in MongoDB. Time: --- 12.687743663787842 secnds ---
Dataframe 003 successfully save in database app-wsr-dev in MongoDB. Time: --- 12.758077144622803 secnds ---
Dataframe 004 successfully save in database app-wsr-dev in MongoDB. Time: --- 12.735713243484497 secnds ---
Dataframe 005 successfully save in database app-wsr-dev in MongoDB. Time: --- 13.040257215499878 secnds ---
Dataframe 006 successfully save in database app-wsr-dev in MongoDB. Time: --- 13.240144491195679 secnds ---
Script execution completed at 28/04/2020 06:49:24. Time: --- 79.49569416046143 secnds ---


In [23]:
#Duplicate to rank countries WORLDWIDE
def get_ww(df): 
    filters = ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    df_ww = df.copy()[(df['ENTITY'].isin(filters))] 
    df_ww['ENTITY'] = "WORLDWIDE"           
    df = df.append(df_ww)
    return df

In [24]:
#Dashboard leaderboard template
def dashboard_leaderboard(df,kpi,domain_num):
    domain = df[(df['KPI'].isin([kpi]))]
    domain = pd.DataFrame({
        'ENTITY': domain['ENTITY_GROUPS'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['ENTITY'],
        'VALUE': domain['VALUE'],
        'VALUE_D-1': domain['VALUE_D-1'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VALUE': domain['UNIT_VALUE'],
        'UNIT_VAR': domain['UNIT_VAR'],
        'UNIT_VARP': domain['UNIT_VARP'],
        'PRECISION_VALUE': domain['PRECISION_VALUE'],
        'PRECISION_VAR': domain['PRECISION_VAR'],
        'PRECISION_VARP': domain['PRECISION_VARP'],
        'LAST_UPDATE': now,
    })
    domain = domain.sort_values(by=['ENTITY','SCENARIO'],ascending=False)
    if domain_num == '010' or domain_num == '011' or domain_num == '012':
        domain = get_ww(domain)
    domain.to_csv(output_folder + domain_num + '.csv',sep=";")
    if use_mongo:
        bob.mongo.save_df(domain,domain_num, db_app,True)
    df_collection = MongoDB[db_app][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING),("SCENARIO", DESCENDING)])
    return domain

In [25]:
# #-- LEADERBOARD BY REGION
# start_time = time.time()
# db_r = db_all[(db_all['ENTITY_GROUPS'] == 'WORLDWIDE') & (db_all['ENTITY'] != 'WORLDWIDE')]
# domain007 = dashboard_leaderboard(db_r,'Confirmed','007')
# domain008 = dashboard_leaderboard(db_r,'Deaths','008')
# domain009 = dashboard_leaderboard(db_r,'Recovered','009')
# print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))

In [26]:
#-- LEADERBOARD BY COUNTRIES
start_time = time.time()
db_c = db_all[(db_all['ENTITY_GROUPS'] != 'WORLDWIDE')]
domain010 = dashboard_leaderboard(db_c,'Confirmed','010')
domain011 = dashboard_leaderboard(db_c,'Deaths','011')
domain012 = dashboard_leaderboard(db_c,'Recovered','012')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))

Dataframe 010 successfully save in database app-wsr-dev in MongoDB. Time: --- 18.541369915008545 secnds ---
Dataframe 011 successfully save in database app-wsr-dev in MongoDB. Time: --- 18.923134803771973 secnds ---
Dataframe 012 successfully save in database app-wsr-dev in MongoDB. Time: --- 19.3862407207489 secnds ---
Script execution completed at 28/04/2020 06:50:23. Time: --- 59.207531213760376 secnds ---


In [12]:
#-- Covid-19 - TRENDS
start_time = time.time()
def domain_101(df,domain_num):
#     filters = ['Confirmed','Deaths','Recovered','Active cases','Deaths ratio','Recovered ratio']
    filters = ['Confirmed','Deaths','Recovered','Active cases']
    domain = df[df['KPI'].isin(filters)]
    domain = pd.DataFrame({
        'ENTITY': domain['ENTITY'],
        'ENTITY_GROUPS': domain['ENTITY_GROUPS'],
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'GROUPS': domain['KPI'],
        'BOTTOM_FILTER': domain['DATE_SCENARIO'],
        'UPPER_FILTER_M': domain['METRIC'],
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
        'LAST_UPDATE': now,
    })

    #Deconfinement France
    groups_name = 'Deconfinement'
    df_france = domain.copy()[(domain['ENTITY'] == 'France') & (domain['GROUPS'] == 'Confirmed') & (domain['UPPER_FILTER_M'] == 'VARV')]
    df_france.loc[:,'GROUPS'] = groups_name
    df_france.loc[:,'VALUE'] = 3000
    domain = pd.concat([domain,df_france],axis=0)
    
    #Rename upper filter
    domain.loc[domain['UPPER_FILTER_M'] == 'VALUE','UPPER_FILTER_M'] = 'EVOLUTION'
    domain.loc[(domain['UPPER_FILTER_M'] == 'VARV') & (domain['GROUPS'].isin(['Confirmed','Deaths','Recovered','Active cases',groups_name])),'UPPER_FILTER_M'] = 'DAILY VARIATION (CASES)'
#     domain.loc[(domain['UPPER_FILTER_M'] == 'VARV') & (domain['GROUPS'].isin(['Deaths ratio','Recovered ratio'])),'UPPER_FILTER_M'] = 'DAILY VARIATION (PTS)'
    domain.loc[(domain['UPPER_FILTER_M'] == 'VARP') & (domain['GROUPS'].isin(['Confirmed','Deaths','Recovered','Active cases',groups_name])),'UPPER_FILTER_M'] = 'DAILY VARIATION (%)'
#     domain.loc[(domain['UPPER_FILTER_M'] == 'VARP') & (domain['GROUPS'].isin(['Deaths ratio','Recovered ratio'])),'UPPER_FILTER_M'] = ''
    
    domain.to_csv(output_folder + domain_num + '.csv',sep=";")
    if use_mongo:
        bob.mongo.save_df(domain,domain_num,db_app,True)
    df_collection = MongoDB[db_app][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING), ("UPPER_FILTER_M", DESCENDING), ("BOTTOM_FILTER", DESCENDING)])
    return domain

domain101 = domain_101(db_trend,'101')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
# domain101

Dataframe 101 successfully save in database app-wsr-dev in MongoDB. Time: --- 64.91215753555298 secnds ---
Script execution completed at 29/04/2020 15:48:05. Time: --- 75.75529313087463 secnds ---


,ENTITY,ENTITY_GROUPS,SCENARIO,DATE_ORDER,GROUPS,BOTTOM_FILTER,UPPER_FILTER_M,VALUE,UNIT_VALUE,PRECISION,LAST_UPDATE
0,Afghanistan,Afghanistan,22/01/2020,20200122,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",29/04/2020 15:38:15
1,Afghanistan,Afghanistan,23/01/2020,20200123,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",29/04/2020 15:38:15
2,Afghanistan,Afghanistan,24/01/2020,20200124,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",29/04/2020 15:38:15
3,Afghanistan,Afghanistan,25/01/2020,20200125,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",29/04/2020 15:38:15
4,Afghanistan,Afghanistan,26/01/2020,20200126,Confirmed,Since beginning,EVOLUTION,0.0,cases,",.0f",29/04/2020 15:38:15
...,...,...,...,...,...,...,...,...,...,...,...
1294389,France,France,01/04/2020,20200401,Deconfinement,Last 30 days,DAILY VARIATION (CASES),3000.0,cases,"+,.0f",29/04/2020 15:38:15
1303639,France,Europe,31/03/2020,20200331,Deconfinement,Last 30 days,DAILY VARIATION (CASES),3000.0,cases,"+,.0f",29/04/2020 15:38:15
1303677,France,France,31/03/2020,20200331,Deconfinement,Last 30 days,DAILY VARIATION (CASES),3000.0,cases,"+,.0f",29/04/2020 15:38:15
1312927,France,Europe,30/03/2020,20200330,Deconfinement,Last 30 days,DAILY VARIATION (CASES),3000.0,cases,"+,.0f",29/04/2020 15:38:15


In [28]:
#-- Covid-19 - TRENDS BY COUNTRIES
start_time = time.time()
def domain_102(df,domain_num):
    domain = df.copy()
    max_date = domain['DATE_ORDER'].max()
    filters =  ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    countries = domain[(domain['ENTITY_GROUPS'].isin(filters)) & (domain['DATE_ORDER'] == max_date) & (domain['GROUPS'] == 'Confirmed') & (domain['BOTTOM_FILTER'] == 'Since beginning')].sort_values(by= 'VALUE', ascending=False)['ENTITY'].drop_duplicates().values.tolist()[:10]
    domain = domain[(domain['ENTITY_GROUPS'] != "WORLDWIDE") & (domain['ENTITY'].isin(countries))]
    
    domain.to_csv(output_folder + domain_num + '.csv',sep=";")
    if use_mongo:
        bob.mongo.save_df(domain,domain_num,db_app,True)
    df_collection = MongoDB[db_app][domain_num]
    df_collection.create_index([("GROUPS", ASCENDING), ("UPPER_FILTER_M", DESCENDING), ("BOTTOM_FILTER", DESCENDING)])
    return domain

domain102 = domain_102(domain101,'102')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
# domain102

Dataframe 102 successfully save in database app-wsr-dev in MongoDB. Time: --- 21.54850196838379 secnds ---
Script execution completed at 28/04/2020 06:51:58. Time: --- 22.135577917099 secnds ---


In [29]:
#-- Covid-19 - RANKING BY COUNTRIES
start_time = time.time()
#-> Referentials
ref_continent = pd.read_excel(input_folder + 'REF_WSR.xlsx', sheet_name = 'REF_CONTINENT')
def domain_103(df,ref,domain_num):
    domain = df.copy()
    cols_to_rename = {"LABEL": "KPI","ENTITY": "LABEL", "ENTITY_GROUPS":"ENTITY", "VARV":"VAR"}
    domain = get_ww(domain.drop(['DATE','UNIT_VAR','UNIT_VARP','PRECISION_VALUE','PRECISION_VAR','PRECISION_VARP'],axis=1).rename(index=str, columns=cols_to_rename).reset_index(drop=True))
    domain = pd.merge(domain, ref, left_on=['LABEL'], right_on=['COUNTRY_REGION'], how='left').drop(['CONTINENT','COUNTRY_REGION'],axis=1).reset_index(drop=True)

    domain.to_csv(output_folder + domain_num + '.csv',sep=";")
    if use_mongo:
        bob.mongo.save_df(domain,domain_num,db_app,True)
    df_collection = MongoDB[db_app][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING), ("SCENARIO", DESCENDING), ("LABEL", DESCENDING), ("KPI", ASCENDING), ("VALUE", DESCENDING)])
    return domain

domain103 = domain_103(db_all,ref_continent,'103')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
# domain103

Dataframe 103 successfully save in database app-wsr-dev in MongoDB. Time: --- 44.13863515853882 secnds ---
Script execution completed at 28/04/2020 06:52:48. Time: --- 50.46979212760925 secnds ---


In [30]:
#-- Covid-19 - DAILY TREND
start_time = time.time()
def domain_106(df,domain_num):
    #-- Exclude Rates
    df = df[~df['KPI'].isin(['Fatality Rate',"Recovery Rate"])]
    
    #-- Calculate Value
    filters = ['WORLDWIDE','Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    domain_val = df.copy()[['ENTITY_GROUPS','ENTITY','SCENARIO','KPI','VALUE','VALUE_D-1']][df['ENTITY_GROUPS'].isin(filters)]#[df['ENTITY_GROUPS'] == 'WORLDWIDE']
    cols_to_keep = ['ENTITY_GROUPS','ENTITY','SCENARIO','KPI']
    domain_val = domain_val.melt(id_vars=cols_to_keep, value_name='VALUE',var_name='METRIC')
    domain_val['ENTITY_GROUPS'] = domain_val['ENTITY']
    domain_val.loc[domain_val['METRIC'] == 'VALUE', 'LABEL_ORDER'] = 99999999
    domain_val.loc[domain_val['METRIC'] == 'VALUE_D-1', 'LABEL_ORDER'] = 0
    domain_val.loc[domain_val['METRIC'] == 'VALUE', 'METRIC'] = domain_val['SCENARIO']
    domain_val.loc[domain_val['METRIC'] == 'VALUE_D-1', 'METRIC'] = (pd.to_datetime(domain_val['SCENARIO'], format='%d/%m/%Y') + timedelta(days=-1)).dt.strftime('%d/%m/%Y')
    
    cols_to_rename = {'METRIC':'LABEL', 'ENTITY_GROUPS': 'ENTITY','KPI':'UPPER_FILTER_R'}
    domain_val = domain_val.drop('ENTITY',axis=1).rename(index=str, columns=cols_to_rename)
    domain_val['PRECISION'] = ',.0f'

    #--Variation
    #filters = ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    domain_var = df.copy()[['ENTITY_GROUPS','ENTITY','SCENARIO','KPI','VARV','VARP']][df['ENTITY_GROUPS']!='WORLDWIDE']#[df['ENTITY_GROUPS'].isin(filters)]
    cols_to_rename = {'VARV':'VALUE', 'VARP': 'VAR'}
    domain_var = domain_var.rename(index=str, columns=cols_to_rename).reset_index(drop=True)
    domain_var = domain_var[domain_var['VALUE'] !=0 ]
    
    #-- Get the first 10
    filters = domain_var[['ENTITY_GROUPS','SCENARIO','KPI']].drop_duplicates().values.tolist()
    domain_w = pd.DataFrame()
    for f in filters:
        tmp_df = domain_var[(domain_var['ENTITY_GROUPS'] == f[0]) & (domain_var['SCENARIO'] == f[1]) & (domain_var['KPI'] == f[2])].sort_values(by=["VALUE","ENTITY"],ascending=False).reset_index(drop=True)
        tmp_df['LABEL_ORDER'] = tmp_df.index.astype(int) + 1
        if len(tmp_df) > 9:
            tmp_df.loc[tmp_df.index > 9, 'ENTITY'] = 'Others'
            tmp_df.loc[tmp_df.index > 9, 'LABEL_ORDER'] = 11
            cols_to_group = ['ENTITY_GROUPS','ENTITY','SCENARIO','KPI','LABEL_ORDER']
            tmp_df = tmp_df.groupby(cols_to_group, as_index=False).agg({'VALUE':'sum'}).reset_index(drop=True)
        tmp_df['PRECISION'] = '+,.0f'
        domain_w = domain_w.append(tmp_df)
        
    cols_to_rename = {'ENTITY':'LABEL', 'ENTITY_GROUPS': 'ENTITY','KPI':'UPPER_FILTER_R'}
    domain_w = domain_w.rename(index=str, columns=cols_to_rename)
    domain_w['GROUPS'] = domain_w['LABEL']
    
    filters = ['Europe','Asia','Africa','North America','South America','Oceania','Other','To be affected']
    cols_to_rename = {'LABEL':'GROUPS', 'ENTITY':'LABEL'}
    domain_w2 = domain_w.copy()[domain_w['ENTITY'].isin(filters)].drop('GROUPS',axis=1).rename(index=str, columns=cols_to_rename)
    domain_w2['ENTITY'] = 'WORLDWIDE'
    
    domain = pd.concat([domain_val,domain_w,domain_w2],axis=0)

    #-- Save df
    domain.to_csv(output_folder + domain_num + '.csv',sep=";")
    if use_mongo:
        bob.mongo.save_df(domain,domain_num,db_app,True)
    df_collection = MongoDB[db_app][domain_num]
    df_collection.create_index([("ENTITY", DESCENDING), ("SCENARIO", DESCENDING), ("UPPER_FILTER_R", ASCENDING)])
    return domain

domain106 = domain_106(db_all,'106')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
# domain106

Dataframe 106 successfully save in database app-wsr-dev in MongoDB. Time: --- 35.09126138687134 secnds ---
Script execution completed at 28/04/2020 07:01:01. Time: --- 492.53746461868286 secnds ---


In [31]:
if use_healthchecks:
    step_2 = hc.done(hc_key)
    print(step_2)

End ==> End ==> send to https://health.cashstory.com/ping/51c8f3e8-3d84-4888-878e-a53f0a152e1a, 2020-04-28
